In [9]:
import xarray as xr
import numpy as np
import pandas as pd
import cfgrib # 用于读取 .grib 文件

# --- 配置数据文件路径 ---
# 请根据您的实际文件路径进行修改
ECMWF_GRIB_PATH = r'E:\ECMWF-S2S\s2s_ecmwf_cf_uvw_pl_step24_2023.grib' # 更新了GRIB文件名

TEST_U_NC_PATH = r'E:\huawei\result\cnn_transformer_test_u_24_5year_order_fusion.nc'
TEST_V_NC_PATH = r'E:\huawei\result\cnn_transformer_testy_v_24_5year_order_fusion.nc'
TEST_W_NC_PATH = r'E:\huawei\result\cnn_transformer_testy_w_24_5year_order_fusion.nc'

PREDICT_U_NC_PATH = r'E:\huawei\result\cnn_transformer_predicty_u_24_5year_order_fusion.nc'
PREDICT_V_NC_PATH = r'E:\huawei\result\cnn_transformer_predicty_v_24_5year_order_fusion.nc'
PREDICT_W_NC_PATH = r'E:\huawei\result\cnn_transformer_predicty_w_24_5year_order_fusion.nc'

# --- Part 1: 加载和预处理数据 ---

print("--- 正在加载和预处理数据 ---")

# 1. 加载ECMWF GRIB数据
try:
    ecmwf_ds = xr.open_dataset(ECMWF_GRIB_PATH, engine='cfgrib')
    print(f"ECMWF数据集已从 {ECMWF_GRIB_PATH} 加载。")
    print("ECMWF数据集的原始维度:", list(ecmwf_ds.dims))
    print("ECMWF数据集的原始变量:", list(ecmwf_ds.data_vars))
except Exception as e:
    print(f"加载ECMWF GRIB文件时出错: {e}")
    print("请确保已安装 'cfgrib' 和 'eccodes' (`pip install cfgrib eccodes`) 且文件路径正确。")
    exit()

# 选取U, V, W分量并存储在字典中，方便后续统一处理
ecmwf_data_arrays = {}

# U分量
if 'u-component_of_wind_isobaric_ens' in ecmwf_ds.data_vars:
    ecmwf_data_arrays['u'] = ecmwf_ds['u-component_of_wind_isobaric_ens']
elif 'u' in ecmwf_ds.data_vars:
    ecmwf_data_arrays['u'] = ecmwf_ds['u']
else:
    print("错误: 在ECMWF数据集中未找到 'u-component_of_wind_isobaric_ens' 或 'u'。")
    print("可用数据变量:", list(ecmwf_ds.data_vars))
    exit()

# V分量
if 'v-component_of_wind_isobaric_ens' in ecmwf_ds.data_vars:
    ecmwf_data_arrays['v'] = ecmwf_ds['v-component_of_wind_isobaric_ens']
elif 'v' in ecmwf_ds.data_vars:
    ecmwf_data_arrays['v'] = ecmwf_ds['v']
else:
    print("错误: 在ECMWF数据集中未找到 'v-component_of_wind_isobaric_ens' 或 'v'。")
    print("可用数据变量:", list(ecmwf_ds.data_vars))
    exit()

# W分量 (垂直速度)
if 'Vertical_velocity_pressure_isobaric_ens' in ecmwf_ds.data_vars:
    ecmwf_data_arrays['w'] = ecmwf_ds['Vertical_velocity_pressure_isobaric_ens']
elif 'w' in ecmwf_ds.data_vars:
    ecmwf_data_arrays['w'] = ecmwf_ds['w']
else:
    print("错误: 在ECMWF数据集中未找到 'Vertical_velocity_pressure_isobaric_ens' 或 'w'。")
    print("可用数据变量:", list(ecmwf_ds.data_vars))
    exit()

# 根据用户反馈，ECMWF数据已经是4维，没有 'ens' 维度，因此无需选择集合成员。
print(f"ECMWF U分量原始维度: {ecmwf_data_arrays['u'].dims}。无需进行集合成员选择。")
print(f"ECMWF V分量原始维度: {ecmwf_data_arrays['v'].dims}。无需进行集合成员选择。")
print(f"ECMWF W分量原始维度: {ecmwf_data_arrays['w'].dims}。无需进行集合成员选择。")


# 识别压力坐标并进行单位转换 (Pa -> hPa)
pressure_coord_name = None
# 假设U,V,W使用相同的压力坐标，取第一个DataArray来识别
for coord_name in ['isobaricInhPa', 'level', 'pressure', 'isobaric']:
    if coord_name in ecmwf_data_arrays['u'].dims:
        pressure_coord_name = coord_name
        break

if pressure_coord_name is None:
    print("错误: 在ECMWF数据中未找到合适的压力坐标。")
    print("可用维度:", list(ecmwf_data_arrays['u'].dims))
    exit()

print(f"ECMWF数据中识别到的压力坐标为: {pressure_coord_name}")

level_units = ecmwf_ds[pressure_coord_name].attrs.get('units') if pressure_coord_name in ecmwf_ds.coords else None
print(f"压力坐标原始单位 (如果可用): {level_units}")

# 对U, V, W分量统一进行压力单位转换、维度重命名和排序
processed_ecmwf_data = {}
for name, da in ecmwf_data_arrays.items():
    if level_units == 'Pa':
        print(f"原始压力单位为 'Pa'。对 {name} 除以100转换为hPa。")
        da = da.rename({pressure_coord_name: 'level_pa_original'})
        da['level'] = da['level_pa_original'] / 100
        da = da.swap_dims({'level_pa_original': 'level'}).drop_vars('level_pa_original')
    elif level_units == 'hPa':
        print(f"原始压力单位为 'hPa'。对 {name} 无需除以100。")
        da = da.rename({pressure_coord_name: 'level'}) # 关键：将压力坐标重命名为 'level'
    else:
        print(f"压力单位未知或不是 'Pa'/'hPa' (检测到: {level_units})。遵循用户明确指令，对 {name} 除以100 (假设原始单位为Pa)。")
        da = da.rename({pressure_coord_name: 'level_original'})
        da['level'] = da['level_original'] / 100
        da = da.swap_dims({'level_original': 'level'}).drop_vars('level_original')

    # 重命名经纬度维度，保持一致性
    da = da.rename({'latitude': 'lat', 'longitude': 'lon'})
    # 确保坐标已排序，以便后续插值操作
    da = da.sortby('level').sortby('lat').sortby('lon')
    processed_ecmwf_data[name] = da

# 将处理后的DataArray重新赋值给ecmwf_u, ecmwf_v, ecmwf_w
ecmwf_u = processed_ecmwf_data['u']
ecmwf_v = processed_ecmwf_data['v']
ecmwf_w = processed_ecmwf_data['w']

print("ECMWF数据预处理完成。最终维度:", ecmwf_u.dims)
print("ECMWF U分量压力层 (hPa):", ecmwf_u['level'].values)


# 2. 加载Test数据
test_u_ds = xr.open_dataset(TEST_U_NC_PATH)
test_v_ds = xr.open_dataset(TEST_V_NC_PATH)
test_w_ds = xr.open_dataset(TEST_W_NC_PATH)

# 将Test数据存储在字典中，方便统一处理
test_data_arrays = {
    'u': test_u_ds['testy_u'],
    'v': test_v_ds['testy_v'],
    'w': test_w_ds['testy_w']
}

# 重命名经纬度维度，保持一致性，并确保坐标已排序
processed_test_data = {}
for name, da in test_data_arrays.items():
    da = da.rename({'longitude': 'lon', 'latitude': 'lat'})
    da = da.sortby('level').sortby('lat').sortby('lon')
    processed_test_data[name] = da

# 将处理后的DataArray重新赋值
test_u = processed_test_data['u']
test_v = processed_test_data['v']
test_w = processed_test_data['w']

print(f"Test U数据已从 {TEST_U_NC_PATH} 加载。维度:", test_u.dims)
print(f"Test V数据已从 {TEST_V_NC_PATH} 加载。维度:", test_v.dims)
print(f"Test W数据已从 {TEST_W_NC_PATH} 加载。维度:", test_w.dims)
print("Test U分量压力层 (hPa):", test_u['level'].values)


# 3. 加载Predict数据
predict_u_ds = xr.open_dataset(PREDICT_U_NC_PATH)
predict_v_ds = xr.open_dataset(PREDICT_V_NC_PATH)
predict_w_ds = xr.open_dataset(PREDICT_W_NC_PATH)

# 将Predict数据存储在字典中，方便统一处理
predict_data_arrays = {
    'u': predict_u_ds['predicty_u'],
    'v': predict_v_ds['predicty_v'],
    'w': predict_w_ds['predicty_w']
}

# 重命名经纬度维度，保持一致性，并确保坐标已排序
processed_predict_data = {}
for name, da in predict_data_arrays.items():
    da = da.rename({'longitude': 'lon', 'latitude': 'lat'})
    da = da.sortby('level').sortby('lat').sortby('lon')
    processed_predict_data[name] = da

# 将处理后的DataArray重新赋值
predict_u = processed_predict_data['u']
predict_v = processed_predict_data['v']
predict_w = processed_predict_data['w']

print(f"Predict U数据已从 {PREDICT_U_NC_PATH} 加载。维度:", predict_u.dims)
print(f"Predict V数据已从 {PREDICT_V_NC_PATH} 加载。维度:", predict_v.dims)
print(f"Predict W数据已从 {PREDICT_W_NC_PATH} 加载。维度:", predict_w.dims)
print("Predict U分量压力层 (hPa):", predict_u['level'].values)


# --- Part 2: 计算ECMWF与Test数据的相关系数和误差指标 ---

print("\n--- 正在计算ECMWF与Test数据的相关系数和误差指标 ---")

# 1. 时间匹配
# 将所有时间坐标转换为datetime64格式，以便进行精确比较
ecmwf_u['time'] = pd.to_datetime(ecmwf_u['time'].values)
ecmwf_v['time'] = pd.to_datetime(ecmwf_v['time'].values)
ecmwf_w['time'] = pd.to_datetime(ecmwf_w['time'].values)
test_u['time'] = pd.to_datetime(test_u['time'].values)
test_v['time'] = pd.to_datetime(test_v['time'].values)
test_w['time'] = pd.to_datetime(test_w['time'].values)

# 找到ECMWF和Test数据集之间共同的时间步
common_times_ecmwf_test = np.intersect1d(
    np.intersect1d(ecmwf_u['time'].values, test_u['time'].values),
    np.intersect1d(ecmwf_v['time'].values, test_v['time'].values)
)
common_times_ecmwf_test = np.intersect1d(common_times_ecmwf_test, np.intersect1d(ecmwf_w['time'].values, test_w['time'].values))


if len(common_times_ecmwf_test) == 0:
    print("错误: ECMWF和Test数据集之间没有找到共同的时间步。无法计算相关性或误差。")
    exit()

# 根据共同时间步选择数据
ecmwf_u_matched_time = ecmwf_u.sel(time=common_times_ecmwf_test)
ecmwf_v_matched_time = ecmwf_v.sel(time=common_times_ecmwf_test)
ecmwf_w_matched_time = ecmwf_w.sel(time=common_times_ecmwf_test)
test_u_matched_time = test_u.sel(time=common_times_ecmwf_test)
test_v_matched_time = test_v.sel(time=common_times_ecmwf_test)
test_w_matched_time = test_w.sel(time=common_times_ecmwf_test)

print(f"ECMWF和Test数据共找到 {len(common_times_ecmwf_test)} 个共同时间步。")

# --- 2. 空间匹配 (以Test的经纬度范围和ECMWF的经纬度分辨率为基准) ---
# 获取Test数据的经纬度范围
test_lat_min = test_u['lat'].min().item()
test_lat_max = test_u['lat'].max().item()
test_lon_min = test_u['lon'].min().item()
test_lon_max = test_u['lon'].max().item()

# 构建目标空间网格：ECMWF的level，以及ECMWF在Test范围内的lat/lon点
target_coords_grid = {
    'level': ecmwf_u_matched_time['level'], # 保持ECMWF的原始高度层
    'lat': ecmwf_u_matched_time['lat'].sel(lat=slice(test_lat_min, test_lat_max)),
    'lon': ecmwf_u_matched_time['lon'].sel(lon=slice(test_lon_min, test_lon_max))
}

# 检查目标网格是否为空
if target_coords_grid['lat'].size == 0 or target_coords_grid['lon'].size == 0:
    print("错误: 目标经纬度网格为空。请检查Test数据和ECMWF数据的经纬度范围是否重叠。")
    exit()

print(f"目标经纬度范围 (基于Test数据): lat [{test_lat_min:.2f}, {test_lat_max:.2f}], lon [{test_lon_min:.2f}, {test_lon_max:.2f}]")
print(f"目标经纬度分辨率 (基于ECMWF数据): lat_res={abs(target_coords_grid['lat'].values[1] - target_coords_grid['lat'].values[0]):.2f}, lon_res={abs(target_coords_grid['lon'].values[1] - target_coords_grid['lon'].values[0]):.2f}")
print(f"目标压力层 (基于ECMWF数据): {target_coords_grid['level'].values}")


# 将ECMWF数据裁剪到目标经纬度范围（保持ECMWF原始高度层和分辨率）
print("正在将ECMWF数据裁剪到目标空间网格 (lat, lon)...")
ecmwf_u_final_for_metrics = ecmwf_u_matched_time.sel(
    lat=target_coords_grid['lat'],
    lon=target_coords_grid['lon']
)
ecmwf_v_final_for_metrics = ecmwf_v_matched_time.sel(
    lat=target_coords_grid['lat'],
    lon=target_coords_grid['lon']
)
ecmwf_w_final_for_metrics = ecmwf_w_matched_time.sel(
    lat=target_coords_grid['lat'],
    lon=target_coords_grid['lon']
)*-0.1

# 将Test数据插值到目标网格 (ECMWF高度层，ECMWF分辨率，Test经纬度范围)
print("正在将Test数据插值到目标空间网格 (level, lat, lon)...")
test_u_matched_spatial = test_u_matched_time.interp(coords=target_coords_grid, method='nearest')
test_v_matched_spatial = test_v_matched_time.interp(coords=target_coords_grid, method='nearest')
test_w_matched_spatial = test_w_matched_time.interp(coords=target_coords_grid, method='nearest')


# 3. 计算相关系数 (ECMWF vs Test)
print("正在计算ECMWF与Test数据的相关系数...")
r_u_ecmwf_test = xr.corr(ecmwf_u_final_for_metrics, test_u_matched_spatial, dim='time')
r_v_ecmwf_test = xr.corr(ecmwf_v_final_for_metrics, test_v_matched_spatial, dim='time')
r_w_ecmwf_test = xr.corr(ecmwf_w_final_for_metrics, test_w_matched_spatial, dim='time')

avg_r_u_ecmwf_test = r_u_ecmwf_test.mean().item()
avg_r_v_ecmwf_test = r_v_ecmwf_test.mean().item()
avg_r_w_ecmwf_test = r_w_ecmwf_test.mean().item()

print(f"\n--- ECMWF与Test数据相关系数结果 ---")
print(f"U分量平均相关系数 (ECMWF vs Test): {avg_r_u_ecmwf_test:.4f}")
print(f"V分量平均相关系数 (ECMWF vs Test): {avg_r_v_ecmwf_test:.4f}")
print(f"W分量平均相关系数 (ECMWF vs Test): {avg_r_w_ecmwf_test:.4f}")


# 4. 计算误差指标 (ECMWF vs Test)
print("\n正在计算ECMWF与Test数据的MAE, RMSE, RMSPE...")

# U分量
diff_u_ecmwf_test = ecmwf_u_final_for_metrics - test_u_matched_spatial
mae_u_ecmwf_test = abs(diff_u_ecmwf_test).mean().item()
rmse_u_ecmwf_test = np.sqrt((diff_u_ecmwf_test**2).mean()).item()
test_u_range = np.nanmax(test_u_matched_spatial.values) - np.nanmin(test_u_matched_spatial.values)
rmspe_u_ecmwf_test = np.nan if test_u_range == 0 else (rmse_u_ecmwf_test / test_u_range)
if test_u_range == 0: print("警告: Test U分量数据范围为零，无法计算RMSPE (ECMWF vs Test)。")

# V分量
diff_v_ecmwf_test = ecmwf_v_final_for_metrics - test_v_matched_spatial
mae_v_ecmwf_test = abs(diff_v_ecmwf_test).mean().item()
rmse_v_ecmwf_test = np.sqrt((diff_v_ecmwf_test**2).mean()).item()
test_v_range = np.nanmax(test_v_matched_spatial.values) - np.nanmin(test_v_matched_spatial.values)
rmspe_v_ecmwf_test = np.nan if test_v_range == 0 else (rmse_v_ecmwf_test / test_v_range)
if test_v_range == 0: print("警告: Test V分量数据范围为零，无法计算RMSPE (ECMWF vs Test)。")

# W分量
diff_w_ecmwf_test = ecmwf_w_final_for_metrics - test_w_matched_spatial
mae_w_ecmwf_test = abs(diff_w_ecmwf_test).mean().item()
rmse_w_ecmwf_test = np.sqrt((diff_w_ecmwf_test**2).mean()).item()
test_w_range = np.nanmax(test_w_matched_spatial.values) - np.nanmin(test_w_matched_spatial.values)
rmspe_w_ecmwf_test = np.nan if test_w_range == 0 else (rmse_w_ecmwf_test / test_w_range)
if test_w_range == 0: print("警告: Test W分量数据范围为零，无法计算RMSPE (ECMWF vs Test)。")


print(f"U分量 MAE (ECMWF vs Test): {mae_u_ecmwf_test:.4f}")
print(f"U分量 RMSE (ECMWF vs Test): {rmse_u_ecmwf_test:.4f}")
print(f"U分量 RMSPE (ECMWF vs Test): {rmspe_u_ecmwf_test:.4f}")
print(f"V分量 MAE (ECMWF vs Test): {mae_v_ecmwf_test:.4f}")
print(f"V分量 RMSE (ECMWF vs Test): {rmse_v_ecmwf_test:.4f}")
print(f"V分量 RMSPE (ECMWF vs Test): {rmspe_v_ecmwf_test:.4f}")
print(f"W分量 MAE (ECMWF vs Test): {mae_w_ecmwf_test:.4f}")
print(f"W分量 RMSE (ECMWF vs Test): {rmse_w_ecmwf_test:.4f}")
print(f"W分量 RMSPE (ECMWF vs Test): {rmspe_w_ecmwf_test:.4f}")


# --- Part 3: 计算Test与Predict数据的相关系数和误差指标 ---

print("\n--- 正在计算Test与Predict数据的相关系数和误差指标 ---")

# 1. 将Predict数据对齐到已匹配的Test数据坐标 (这些坐标基于ECMWF)
predict_u['time'] = pd.to_datetime(predict_u['time'].values)
predict_v['time'] = pd.to_datetime(predict_v['time'].values)
predict_w['time'] = pd.to_datetime(predict_w['time'].values)

# 注意：这里使用 test_u_matched_spatial 的时间，因为它已经与ECMWF时间对齐
common_times_predict_test = np.intersect1d(
    np.intersect1d(predict_u['time'].values, test_u_matched_spatial['time'].values),
    np.intersect1d(predict_v['time'].values, test_v_matched_spatial['time'].values)
)
common_times_predict_test = np.intersect1d(common_times_predict_test, np.intersect1d(predict_w['time'].values, test_w_matched_spatial['time'].values))


if len(common_times_predict_test) == 0:
    print("错误: Predict数据和已匹配的Test数据集之间没有找到共同的时间步。无法计算相关性或误差。")
    exit()

# 根据共同时间步选择Predict数据
predict_u_matched_time = predict_u.sel(time=common_times_predict_test)
predict_v_matched_time = predict_v.sel(time=common_times_predict_test)
predict_w_matched_time = predict_w.sel(time=common_times_predict_test)

# 同时，将之前已匹配的Test数据也过滤到这些共同时间步
test_u_final_for_predict_corr = test_u_matched_spatial.sel(time=common_times_predict_test)
test_v_final_for_predict_corr = test_v_matched_spatial.sel(time=common_times_predict_test)
test_w_final_for_predict_corr = test_w_matched_spatial.sel(time=common_times_predict_test)

print(f"Predict数据和 (已匹配的) Test数据共找到 {len(common_times_predict_test)} 个共同时间步。")

# 将Predict数据插值到之前定义的目标网格 (target_coords_grid)
print("正在将Predict数据插值到共同的空间网格...")
predict_u_final_for_metrics = predict_u_matched_time.interp(coords=target_coords_grid, method='nearest')
predict_v_final_for_metrics = predict_v_matched_time.interp(coords=target_coords_grid, method='nearest')
predict_w_final_for_metrics = predict_w_matched_time.interp(coords=target_coords_grid, method='nearest')

# 2. 计算相关系数 (Test vs Predict)
print("正在计算Test与Predict数据的相关系数...")
r_u_test_predict = xr.corr(test_u_final_for_predict_corr, predict_u_final_for_metrics, dim='time')
r_v_test_predict = xr.corr(test_v_final_for_predict_corr, predict_v_final_for_metrics, dim='time')
r_w_test_predict = xr.corr(test_w_final_for_predict_corr, predict_w_final_for_metrics, dim='time')

avg_r_u_test_predict = r_u_test_predict.mean().item()
avg_r_v_test_predict = r_v_test_predict.mean().item()
avg_r_w_test_predict = r_w_test_predict.mean().item()

print(f"\n--- Test与Predict数据相关系数结果 ---")
print(f"U分量平均相关系数 (Test vs Predict): {avg_r_u_test_predict:.4f}")
print(f"V分量平均相关系数 (Test vs Predict): {avg_r_v_test_predict:.4f}")
print(f"W分量平均相关系数 (Test vs Predict): {avg_r_w_test_predict:.4f}")


# 3. 计算误差指标 (Test vs Predict)
print("\n正在计算Test与Predict数据的MAE, RMSE, RMSPE...")

# U分量
diff_u_test_predict = predict_u_final_for_metrics - test_u_final_for_predict_corr
mae_u_test_predict = abs(diff_u_test_predict).mean().item()
rmse_u_test_predict = np.sqrt((diff_u_test_predict**2).mean()).item()
test_u_range_predict = np.nanmax(test_u_final_for_predict_corr.values) - np.nanmin(test_u_final_for_predict_corr.values)
rmspe_u_test_predict = np.nan if test_u_range_predict == 0 else (rmse_u_test_predict / test_u_range_predict)
if test_u_range_predict == 0: print("警告: Test U分量数据范围为零，无法计算RMSPE (Test vs Predict)。")

# V分量
diff_v_test_predict = predict_v_final_for_metrics - test_v_final_for_predict_corr
mae_v_test_predict = abs(diff_v_test_predict).mean().item()
rmse_v_test_predict = np.sqrt((diff_v_test_predict**2).mean()).item()
test_v_range_predict = np.nanmax(test_v_final_for_predict_corr.values) - np.nanmin(test_v_final_for_predict_corr.values)
rmspe_v_test_predict = np.nan if test_v_range_predict == 0 else (rmse_v_test_predict / test_v_range_predict)
if test_v_range_predict == 0: print("警告: Test V分量数据范围为零，无法计算RMSPE (Test vs Predict)。")

# W分量
diff_w_test_predict = predict_w_final_for_metrics - test_w_final_for_predict_corr
mae_w_test_predict = abs(diff_w_test_predict).mean().item()
rmse_w_test_predict = np.sqrt((diff_w_test_predict**2).mean()).item()
test_w_range_predict = np.nanmax(test_w_final_for_predict_corr.values) - np.nanmin(test_w_final_for_predict_corr.values)
rmspe_w_test_predict = np.nan if test_w_range_predict == 0 else (rmse_w_test_predict / test_w_range_predict)
if test_w_range_predict == 0: print("警告: Test W分量数据范围为零，无法计算RMSPE (Test vs Predict)。")


print(f"U分量 MAE (Test vs Predict): {mae_u_test_predict:.4f}")
print(f"U分量 RMSE (Test vs Predict): {rmse_u_test_predict:.4f}")
print(f"U分量 RMSPE (Test vs Predict): {rmspe_u_test_predict:.4f}")
print(f"V分量 MAE (Test vs Predict): {mae_v_test_predict:.4f}")
print(f"V分量 RMSE (Test vs Predict): {rmse_v_test_predict:.4f}")
print(f"V分量 RMSPE (Test vs Predict): {rmspe_v_test_predict:.4f}")
print(f"W分量 MAE (Test vs Predict): {mae_w_test_predict:.4f}")
print(f"W分量 RMSE (Test vs Predict): {rmse_w_test_predict:.4f}")
print(f"W分量 RMSPE (Test vs Predict): {rmspe_w_test_predict:.4f}")

print("\n--- 脚本执行完毕 ---")

--- 正在加载和预处理数据 ---


C:\Users\TBYC\AppData\Roaming\Python\Python311\site-packages\cfgrib\xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(


ECMWF数据集已从 E:\ECMWF-S2S\s2s_ecmwf_cf_uvw_pl_step24_2023.grib 加载。
ECMWF数据集的原始维度: ['time', 'isobaricInhPa', 'latitude', 'longitude']
ECMWF数据集的原始变量: ['u', 'v', 'w']
ECMWF U分量原始维度: ('time', 'isobaricInhPa', 'latitude', 'longitude')。无需进行集合成员选择。
ECMWF V分量原始维度: ('time', 'isobaricInhPa', 'latitude', 'longitude')。无需进行集合成员选择。
ECMWF W分量原始维度: ('time', 'isobaricInhPa', 'latitude', 'longitude')。无需进行集合成员选择。
ECMWF数据中识别到的压力坐标为: isobaricInhPa
压力坐标原始单位 (如果可用): hPa
原始压力单位为 'hPa'。对 u 无需除以100。
原始压力单位为 'hPa'。对 v 无需除以100。
原始压力单位为 'hPa'。对 w 无需除以100。
ECMWF数据预处理完成。最终维度: ('time', 'level', 'lat', 'lon')
ECMWF U分量压力层 (hPa): [ 500.  700.  850.  925. 1000.]
Test U数据已从 E:\huawei\result\cnn_transformer_test_u_24_5year_order_fusion.nc 加载。维度: ('time', 'level', 'lat', 'lon')
Test V数据已从 E:\huawei\result\cnn_transformer_testy_v_24_5year_order_fusion.nc 加载。维度: ('time', 'level', 'lat', 'lon')
Test W数据已从 E:\huawei\result\cnn_transformer_testy_w_24_5year_order_fusion.nc 加载。维度: ('time', 'level', 'lat', 'lon')
Test U分量压力层 (hPa): 

In [10]:
import xarray as xr
import numpy as np
import pandas as pd
import cfgrib # 用于读取 .grib 文件

# --- 配置数据文件路径 ---
# 请根据您的实际文件路径进行修改
ECMWF_GRIB_PATH = r'E:\ECMWF-S2S\s2s_ecmwf_cf_uvw_pl_step0_2023.grib' # 更新了GRIB文件名

TEST_U_NC_PATH = r'E:\huawei\result\cnn_transformer_test_u_24_5year_order_fusion.nc'
TEST_V_NC_PATH = r'E:\huawei\result\cnn_transformer_testy_v_24_5year_order_fusion.nc'
TEST_W_NC_PATH = r'E:\huawei\result\cnn_transformer_testy_w_24_5year_order_fusion.nc'

PREDICT_U_NC_PATH = r'E:\huawei\result\cnn_transformer_predicty_u_24_5year_order_fusion.nc'
PREDICT_V_NC_PATH = r'E:\huawei\result\cnn_transformer_predicty_v_24_5year_order_fusion.nc'
PREDICT_W_NC_PATH = r'E:\huawei\result\cnn_transformer_predicty_w_24_5year_order_fusion.nc'

# --- Part 1: 加载和预处理数据 ---

print("--- 正在加载和预处理数据 ---")

# 1. 加载ECMWF GRIB数据
try:
    ecmwf_ds = xr.open_dataset(ECMWF_GRIB_PATH, engine='cfgrib')
    print(f"ECMWF数据集已从 {ECMWF_GRIB_PATH} 加载。")
    print("ECMWF数据集的原始维度:", list(ecmwf_ds.dims))
    print("ECMWF数据集的原始变量:", list(ecmwf_ds.data_vars))
except Exception as e:
    print(f"加载ECMWF GRIB文件时出错: {e}")
    print("请确保已安装 'cfgrib' 和 'eccodes' (`pip install cfgrib eccodes`) 且文件路径正确。")
    exit()

# 选取U, V, W分量并存储在字典中，方便后续统一处理
ecmwf_data_arrays = {}

# U分量
if 'u-component_of_wind_isobaric_ens' in ecmwf_ds.data_vars:
    ecmwf_data_arrays['u'] = ecmwf_ds['u-component_of_wind_isobaric_ens']
elif 'u' in ecmwf_ds.data_vars:
    ecmwf_data_arrays['u'] = ecmwf_ds['u']
else:
    print("错误: 在ECMWF数据集中未找到 'u-component_of_wind_isobaric_ens' 或 'u'。")
    print("可用数据变量:", list(ecmwf_ds.data_vars))
    exit()

# V分量
if 'v-component_of_wind_isobaric_ens' in ecmwf_ds.data_vars:
    ecmwf_data_arrays['v'] = ecmwf_ds['v-component_of_wind_isobaric_ens']
elif 'v' in ecmwf_ds.data_vars:
    ecmwf_data_arrays['v'] = ecmwf_ds['v']
else:
    print("错误: 在ECMWF数据集中未找到 'v-component_of_wind_isobaric_ens' 或 'v'。")
    print("可用数据变量:", list(ecmwf_ds.data_vars))
    exit()

# W分量 (垂直速度)
if 'Vertical_velocity_pressure_isobaric_ens' in ecmwf_ds.data_vars:
    ecmwf_data_arrays['w'] = ecmwf_ds['Vertical_velocity_pressure_isobaric_ens']
elif 'w' in ecmwf_ds.data_vars:
    ecmwf_data_arrays['w'] = ecmwf_ds['w']
else:
    print("错误: 在ECMWF数据集中未找到 'Vertical_velocity_pressure_isobaric_ens' 或 'w'。")
    print("可用数据变量:", list(ecmwf_ds.data_vars))
    exit()

# 根据用户反馈，ECMWF数据已经是4维，没有 'ens' 维度，因此无需选择集合成员。
print(f"ECMWF U分量原始维度: {ecmwf_data_arrays['u'].dims}。无需进行集合成员选择。")
print(f"ECMWF V分量原始维度: {ecmwf_data_arrays['v'].dims}。无需进行集合成员选择。")
print(f"ECMWF W分量原始维度: {ecmwf_data_arrays['w'].dims}。无需进行集合成员选择。")


# 识别压力坐标并进行单位转换 (Pa -> hPa)
pressure_coord_name = None
# 假设U,V,W使用相同的压力坐标，取第一个DataArray来识别
for coord_name in ['isobaricInhPa', 'level', 'pressure', 'isobaric']:
    if coord_name in ecmwf_data_arrays['u'].dims:
        pressure_coord_name = coord_name
        break

if pressure_coord_name is None:
    print("错误: 在ECMWF数据中未找到合适的压力坐标。")
    print("可用维度:", list(ecmwf_data_arrays['u'].dims))
    exit()

print(f"ECMWF数据中识别到的压力坐标为: {pressure_coord_name}")

level_units = ecmwf_ds[pressure_coord_name].attrs.get('units') if pressure_coord_name in ecmwf_ds.coords else None
print(f"压力坐标原始单位 (如果可用): {level_units}")

# 对U, V, W分量统一进行压力单位转换、维度重命名和排序
processed_ecmwf_data = {}
for name, da in ecmwf_data_arrays.items():
    if level_units == 'Pa':
        print(f"原始压力单位为 'Pa'。对 {name} 除以100转换为hPa。")
        da = da.rename({pressure_coord_name: 'level_pa_original'})
        da['level'] = da['level_pa_original'] / 100
        da = da.swap_dims({'level_pa_original': 'level'}).drop_vars('level_pa_original')
    elif level_units == 'hPa':
        print(f"原始压力单位为 'hPa'。对 {name} 无需除以100。")
        da = da.rename({pressure_coord_name: 'level'}) # 关键：将压力坐标重命名为 'level'
    else:
        print(f"压力单位未知或不是 'Pa'/'hPa' (检测到: {level_units})。遵循用户明确指令，对 {name} 除以100 (假设原始单位为Pa)。")
        da = da.rename({pressure_coord_name: 'level_original'})
        da['level'] = da['level_original'] / 100
        da = da.swap_dims({'level_original': 'level'}).drop_vars('level_original')

    # 重命名经纬度维度，保持一致性
    da = da.rename({'latitude': 'lat', 'longitude': 'lon'})
    # 确保坐标已排序，以便后续插值操作
    da = da.sortby('level').sortby('lat').sortby('lon')
    processed_ecmwf_data[name] = da

# 将处理后的DataArray重新赋值给ecmwf_u, ecmwf_v, ecmwf_w
ecmwf_u = processed_ecmwf_data['u']
ecmwf_v = processed_ecmwf_data['v']
ecmwf_w = processed_ecmwf_data['w']

print("ECMWF数据预处理完成。最终维度:", ecmwf_u.dims)
print("ECMWF U分量压力层 (hPa):", ecmwf_u['level'].values)


# 2. 加载Test数据
test_u_ds = xr.open_dataset(TEST_U_NC_PATH)
test_v_ds = xr.open_dataset(TEST_V_NC_PATH)
test_w_ds = xr.open_dataset(TEST_W_NC_PATH)

# 将Test数据存储在字典中，方便统一处理
test_data_arrays = {
    'u': test_u_ds['testy_u'],
    'v': test_v_ds['testy_v'],
    'w': test_w_ds['testy_w']
}

# 重命名经纬度维度，保持一致性，并确保坐标已排序
processed_test_data = {}
for name, da in test_data_arrays.items():
    da = da.rename({'longitude': 'lon', 'latitude': 'lat'})
    da = da.sortby('level').sortby('lat').sortby('lon')
    processed_test_data[name] = da

# 将处理后的DataArray重新赋值
test_u = processed_test_data['u']
test_v = processed_test_data['v']
test_w = processed_test_data['w']

print(f"Test U数据已从 {TEST_U_NC_PATH} 加载。维度:", test_u.dims)
print(f"Test V数据已从 {TEST_V_NC_PATH} 加载。维度:", test_v.dims)
print(f"Test W数据已从 {TEST_W_NC_PATH} 加载。维度:", test_w.dims)
print("Test U分量压力层 (hPa):", test_u['level'].values)


# 3. 加载Predict数据
predict_u_ds = xr.open_dataset(PREDICT_U_NC_PATH)
predict_v_ds = xr.open_dataset(PREDICT_V_NC_PATH)
predict_w_ds = xr.open_dataset(PREDICT_W_NC_PATH)

# 将Predict数据存储在字典中，方便统一处理
predict_data_arrays = {
    'u': predict_u_ds['predicty_u'],
    'v': predict_v_ds['predicty_v'],
    'w': predict_w_ds['predicty_w']
}

# 重命名经纬度维度，保持一致性，并确保坐标已排序
processed_predict_data = {}
for name, da in predict_data_arrays.items():
    da = da.rename({'longitude': 'lon', 'latitude': 'lat'})
    da = da.sortby('level').sortby('lat').sortby('lon')
    processed_predict_data[name] = da

# 将处理后的DataArray重新赋值
predict_u = processed_predict_data['u']
predict_v = processed_predict_data['v']
predict_w = processed_predict_data['w']

print(f"Predict U数据已从 {PREDICT_U_NC_PATH} 加载。维度:", predict_u.dims)
print(f"Predict V数据已从 {PREDICT_V_NC_PATH} 加载。维度:", predict_v.dims)
print(f"Predict W数据已从 {PREDICT_W_NC_PATH} 加载。维度:", predict_w.dims)
print("Predict U分量压力层 (hPa):", predict_u['level'].values)


# --- Part 2: 计算ECMWF与Test数据的相关系数和误差指标 ---

print("\n--- 正在计算ECMWF与Test数据的相关系数和误差指标 ---")

# 1. 时间匹配
# 将所有时间坐标转换为datetime64格式，以便进行精确比较
ecmwf_u['time'] = pd.to_datetime(ecmwf_u['time'].values)
ecmwf_v['time'] = pd.to_datetime(ecmwf_v['time'].values)
ecmwf_w['time'] = pd.to_datetime(ecmwf_w['time'].values)
test_u['time'] = pd.to_datetime(test_u['time'].values)
test_v['time'] = pd.to_datetime(test_v['time'].values)
test_w['time'] = pd.to_datetime(test_w['time'].values)

# 找到ECMWF和Test数据集之间共同的时间步
common_times_ecmwf_test = np.intersect1d(
    np.intersect1d(ecmwf_u['time'].values, test_u['time'].values),
    np.intersect1d(ecmwf_v['time'].values, test_v['time'].values)
)
common_times_ecmwf_test = np.intersect1d(common_times_ecmwf_test, np.intersect1d(ecmwf_w['time'].values, test_w['time'].values))


if len(common_times_ecmwf_test) == 0:
    print("错误: ECMWF和Test数据集之间没有找到共同的时间步。无法计算相关性或误差。")
    exit()

# 根据共同时间步选择数据
ecmwf_u_matched_time = ecmwf_u.sel(time=common_times_ecmwf_test)
ecmwf_v_matched_time = ecmwf_v.sel(time=common_times_ecmwf_test)
ecmwf_w_matched_time = ecmwf_w.sel(time=common_times_ecmwf_test)
test_u_matched_time = test_u.sel(time=common_times_ecmwf_test)
test_v_matched_time = test_v.sel(time=common_times_ecmwf_test)
test_w_matched_time = test_w.sel(time=common_times_ecmwf_test)

print(f"ECMWF和Test数据共找到 {len(common_times_ecmwf_test)} 个共同时间步。")

# --- 2. 空间匹配 (以Test的经纬度范围和ECMWF的经纬度分辨率为基准) ---
# 获取Test数据的经纬度范围
test_lat_min = test_u['lat'].min().item()
test_lat_max = test_u['lat'].max().item()
test_lon_min = test_u['lon'].min().item()
test_lon_max = test_u['lon'].max().item()

# 构建目标空间网格：ECMWF的level，以及ECMWF在Test范围内的lat/lon点
target_coords_grid = {
    'level': ecmwf_u_matched_time['level'], # 保持ECMWF的原始高度层
    'lat': ecmwf_u_matched_time['lat'].sel(lat=slice(test_lat_min, test_lat_max)),
    'lon': ecmwf_u_matched_time['lon'].sel(lon=slice(test_lon_min, test_lon_max))
}

# 检查目标网格是否为空
if target_coords_grid['lat'].size == 0 or target_coords_grid['lon'].size == 0:
    print("错误: 目标经纬度网格为空。请检查Test数据和ECMWF数据的经纬度范围是否重叠。")
    exit()

print(f"目标经纬度范围 (基于Test数据): lat [{test_lat_min:.2f}, {test_lat_max:.2f}], lon [{test_lon_min:.2f}, {test_lon_max:.2f}]")
print(f"目标经纬度分辨率 (基于ECMWF数据): lat_res={abs(target_coords_grid['lat'].values[1] - target_coords_grid['lat'].values[0]):.2f}, lon_res={abs(target_coords_grid['lon'].values[1] - target_coords_grid['lon'].values[0]):.2f}")
print(f"目标压力层 (基于ECMWF数据): {target_coords_grid['level'].values}")


# 将ECMWF数据裁剪到目标经纬度范围（保持ECMWF原始高度层和分辨率）
print("正在将ECMWF数据裁剪到目标空间网格 (lat, lon)...")
ecmwf_u_final_for_metrics = ecmwf_u_matched_time.sel(
    lat=target_coords_grid['lat'],
    lon=target_coords_grid['lon']
)
ecmwf_v_final_for_metrics = ecmwf_v_matched_time.sel(
    lat=target_coords_grid['lat'],
    lon=target_coords_grid['lon']
)
ecmwf_w_final_for_metrics = ecmwf_w_matched_time.sel(
    lat=target_coords_grid['lat'],
    lon=target_coords_grid['lon']
)*-0.1

# 将Test数据插值到目标网格 (ECMWF高度层，ECMWF分辨率，Test经纬度范围)
print("正在将Test数据插值到目标空间网格 (level, lat, lon)...")
test_u_matched_spatial = test_u_matched_time.interp(coords=target_coords_grid, method='nearest')
test_v_matched_spatial = test_v_matched_time.interp(coords=target_coords_grid, method='nearest')
test_w_matched_spatial = test_w_matched_time.interp(coords=target_coords_grid, method='nearest')


# 3. 计算相关系数 (ECMWF vs Test)
print("正在计算ECMWF与Test数据的相关系数...")
r_u_ecmwf_test = xr.corr(ecmwf_u_final_for_metrics, test_u_matched_spatial, dim='time')
r_v_ecmwf_test = xr.corr(ecmwf_v_final_for_metrics, test_v_matched_spatial, dim='time')
r_w_ecmwf_test = xr.corr(ecmwf_w_final_for_metrics, test_w_matched_spatial, dim='time')

avg_r_u_ecmwf_test = r_u_ecmwf_test.mean().item()
avg_r_v_ecmwf_test = r_v_ecmwf_test.mean().item()
avg_r_w_ecmwf_test = r_w_ecmwf_test.mean().item()

print(f"\n--- ECMWF与Test数据相关系数结果 ---")
print(f"U分量平均相关系数 (ECMWF vs Test): {avg_r_u_ecmwf_test:.4f}")
print(f"V分量平均相关系数 (ECMWF vs Test): {avg_r_v_ecmwf_test:.4f}")
print(f"W分量平均相关系数 (ECMWF vs Test): {avg_r_w_ecmwf_test:.4f}")


# 4. 计算误差指标 (ECMWF vs Test)
print("\n正在计算ECMWF与Test数据的MAE, RMSE, RMSPE...")

# U分量
diff_u_ecmwf_test = ecmwf_u_final_for_metrics - test_u_matched_spatial
mae_u_ecmwf_test = abs(diff_u_ecmwf_test).mean().item()
rmse_u_ecmwf_test = np.sqrt((diff_u_ecmwf_test**2).mean()).item()
test_u_range = np.nanmax(test_u_matched_spatial.values) - np.nanmin(test_u_matched_spatial.values)
rmspe_u_ecmwf_test = np.nan if test_u_range == 0 else (rmse_u_ecmwf_test / test_u_range)
if test_u_range == 0: print("警告: Test U分量数据范围为零，无法计算RMSPE (ECMWF vs Test)。")

# V分量
diff_v_ecmwf_test = ecmwf_v_final_for_metrics - test_v_matched_spatial
mae_v_ecmwf_test = abs(diff_v_ecmwf_test).mean().item()
rmse_v_ecmwf_test = np.sqrt((diff_v_ecmwf_test**2).mean()).item()
test_v_range = np.nanmax(test_v_matched_spatial.values) - np.nanmin(test_v_matched_spatial.values)
rmspe_v_ecmwf_test = np.nan if test_v_range == 0 else (rmse_v_ecmwf_test / test_v_range)
if test_v_range == 0: print("警告: Test V分量数据范围为零，无法计算RMSPE (ECMWF vs Test)。")

# W分量
diff_w_ecmwf_test = ecmwf_w_final_for_metrics - test_w_matched_spatial
mae_w_ecmwf_test = abs(diff_w_ecmwf_test).mean().item()
rmse_w_ecmwf_test = np.sqrt((diff_w_ecmwf_test**2).mean()).item()
test_w_range = np.nanmax(test_w_matched_spatial.values) - np.nanmin(test_w_matched_spatial.values)
rmspe_w_ecmwf_test = np.nan if test_w_range == 0 else (rmse_w_ecmwf_test / test_w_range)
if test_w_range == 0: print("警告: Test W分量数据范围为零，无法计算RMSPE (ECMWF vs Test)。")


print(f"U分量 MAE (ECMWF vs Test): {mae_u_ecmwf_test:.4f}")
print(f"U分量 RMSE (ECMWF vs Test): {rmse_u_ecmwf_test:.4f}")
print(f"U分量 RMSPE (ECMWF vs Test): {rmspe_u_ecmwf_test:.4f}")
print(f"V分量 MAE (ECMWF vs Test): {mae_v_ecmwf_test:.4f}")
print(f"V分量 RMSE (ECMWF vs Test): {rmse_v_ecmwf_test:.4f}")
print(f"V分量 RMSPE (ECMWF vs Test): {rmspe_v_ecmwf_test:.4f}")
print(f"W分量 MAE (ECMWF vs Test): {mae_w_ecmwf_test:.4f}")
print(f"W分量 RMSE (ECMWF vs Test): {rmse_w_ecmwf_test:.4f}")
print(f"W分量 RMSPE (ECMWF vs Test): {rmspe_w_ecmwf_test:.4f}")


# --- Part 3: 计算Test与Predict数据的相关系数和误差指标 ---

print("\n--- 正在计算Test与Predict数据的相关系数和误差指标 ---")

# 1. 将Predict数据对齐到已匹配的Test数据坐标 (这些坐标基于ECMWF)
predict_u['time'] = pd.to_datetime(predict_u['time'].values)
predict_v['time'] = pd.to_datetime(predict_v['time'].values)
predict_w['time'] = pd.to_datetime(predict_w['time'].values)

# 注意：这里使用 test_u_matched_spatial 的时间，因为它已经与ECMWF时间对齐
common_times_predict_test = np.intersect1d(
    np.intersect1d(predict_u['time'].values, test_u_matched_spatial['time'].values),
    np.intersect1d(predict_v['time'].values, test_v_matched_spatial['time'].values)
)
common_times_predict_test = np.intersect1d(common_times_predict_test, np.intersect1d(predict_w['time'].values, test_w_matched_spatial['time'].values))


if len(common_times_predict_test) == 0:
    print("错误: Predict数据和已匹配的Test数据集之间没有找到共同的时间步。无法计算相关性或误差。")
    exit()

# 根据共同时间步选择Predict数据
predict_u_matched_time = predict_u.sel(time=common_times_predict_test)
predict_v_matched_time = predict_v.sel(time=common_times_predict_test)
predict_w_matched_time = predict_w.sel(time=common_times_predict_test)

# 同时，将之前已匹配的Test数据也过滤到这些共同时间步
test_u_final_for_predict_corr = test_u_matched_spatial.sel(time=common_times_predict_test)
test_v_final_for_predict_corr = test_v_matched_spatial.sel(time=common_times_predict_test)
test_w_final_for_predict_corr = test_w_matched_spatial.sel(time=common_times_predict_test)

print(f"Predict数据和 (已匹配的) Test数据共找到 {len(common_times_predict_test)} 个共同时间步。")

# 将Predict数据插值到之前定义的目标网格 (target_coords_grid)
print("正在将Predict数据插值到共同的空间网格...")
predict_u_final_for_metrics = predict_u_matched_time.interp(coords=target_coords_grid, method='nearest')
predict_v_final_for_metrics = predict_v_matched_time.interp(coords=target_coords_grid, method='nearest')
predict_w_final_for_metrics = predict_w_matched_time.interp(coords=target_coords_grid, method='nearest')

# 2. 计算相关系数 (Test vs Predict)
print("正在计算Test与Predict数据的相关系数...")
r_u_test_predict = xr.corr(test_u_final_for_predict_corr, predict_u_final_for_metrics, dim='time')
r_v_test_predict = xr.corr(test_v_final_for_predict_corr, predict_v_final_for_metrics, dim='time')
r_w_test_predict = xr.corr(test_w_final_for_predict_corr, predict_w_final_for_metrics, dim='time')

avg_r_u_test_predict = r_u_test_predict.mean().item()
avg_r_v_test_predict = r_v_test_predict.mean().item()
avg_r_w_test_predict = r_w_test_predict.mean().item()

print(f"\n--- Test与Predict数据相关系数结果 ---")
print(f"U分量平均相关系数 (Test vs Predict): {avg_r_u_test_predict:.4f}")
print(f"V分量平均相关系数 (Test vs Predict): {avg_r_v_test_predict:.4f}")
print(f"W分量平均相关系数 (Test vs Predict): {avg_r_w_test_predict:.4f}")


# 3. 计算误差指标 (Test vs Predict)
print("\n正在计算Test与Predict数据的MAE, RMSE, RMSPE...")

# U分量
diff_u_test_predict = predict_u_final_for_metrics - test_u_final_for_predict_corr
mae_u_test_predict = abs(diff_u_test_predict).mean().item()
rmse_u_test_predict = np.sqrt((diff_u_test_predict**2).mean()).item()
test_u_range_predict = np.nanmax(test_u_final_for_predict_corr.values) - np.nanmin(test_u_final_for_predict_corr.values)
rmspe_u_test_predict = np.nan if test_u_range_predict == 0 else (rmse_u_test_predict / test_u_range_predict)
if test_u_range_predict == 0: print("警告: Test U分量数据范围为零，无法计算RMSPE (Test vs Predict)。")

# V分量
diff_v_test_predict = predict_v_final_for_metrics - test_v_final_for_predict_corr
mae_v_test_predict = abs(diff_v_test_predict).mean().item()
rmse_v_test_predict = np.sqrt((diff_v_test_predict**2).mean()).item()
test_v_range_predict = np.nanmax(test_v_final_for_predict_corr.values) - np.nanmin(test_v_final_for_predict_corr.values)
rmspe_v_test_predict = np.nan if test_v_range_predict == 0 else (rmse_v_test_predict / test_v_range_predict)
if test_v_range_predict == 0: print("警告: Test V分量数据范围为零，无法计算RMSPE (Test vs Predict)。")

# W分量
diff_w_test_predict = predict_w_final_for_metrics - test_w_final_for_predict_corr
mae_w_test_predict = abs(diff_w_test_predict).mean().item()
rmse_w_test_predict = np.sqrt((diff_w_test_predict**2).mean()).item()
test_w_range_predict = np.nanmax(test_w_final_for_predict_corr.values) - np.nanmin(test_w_final_for_predict_corr.values)
rmspe_w_test_predict = np.nan if test_w_range_predict == 0 else (rmse_w_test_predict / test_w_range_predict)
if test_w_range_predict == 0: print("警告: Test W分量数据范围为零，无法计算RMSPE (Test vs Predict)。")


print(f"U分量 MAE (Test vs Predict): {mae_u_test_predict:.4f}")
print(f"U分量 RMSE (Test vs Predict): {rmse_u_test_predict:.4f}")
print(f"U分量 RMSPE (Test vs Predict): {rmspe_u_test_predict:.4f}")
print(f"V分量 MAE (Test vs Predict): {mae_v_test_predict:.4f}")
print(f"V分量 RMSE (Test vs Predict): {rmse_v_test_predict:.4f}")
print(f"V分量 RMSPE (Test vs Predict): {rmspe_v_test_predict:.4f}")
print(f"W分量 MAE (Test vs Predict): {mae_w_test_predict:.4f}")
print(f"W分量 RMSE (Test vs Predict): {rmse_w_test_predict:.4f}")
print(f"W分量 RMSPE (Test vs Predict): {rmspe_w_test_predict:.4f}")

print("\n--- 脚本执行完毕 ---")

--- 正在加载和预处理数据 ---
ECMWF数据集已从 E:\ECMWF-S2S\s2s_ecmwf_cf_uvw_pl_step0_2023.grib 加载。
ECMWF数据集的原始维度: ['time', 'isobaricInhPa', 'latitude', 'longitude']
ECMWF数据集的原始变量: ['u', 'v', 'w']
ECMWF U分量原始维度: ('time', 'isobaricInhPa', 'latitude', 'longitude')。无需进行集合成员选择。
ECMWF V分量原始维度: ('time', 'isobaricInhPa', 'latitude', 'longitude')。无需进行集合成员选择。
ECMWF W分量原始维度: ('time', 'isobaricInhPa', 'latitude', 'longitude')。无需进行集合成员选择。
ECMWF数据中识别到的压力坐标为: isobaricInhPa
压力坐标原始单位 (如果可用): hPa
原始压力单位为 'hPa'。对 u 无需除以100。
原始压力单位为 'hPa'。对 v 无需除以100。
原始压力单位为 'hPa'。对 w 无需除以100。
ECMWF数据预处理完成。最终维度: ('time', 'level', 'lat', 'lon')
ECMWF U分量压力层 (hPa): [ 500.  700.  850.  925. 1000.]
Test U数据已从 E:\huawei\result\cnn_transformer_test_u_24_5year_order_fusion.nc 加载。维度: ('time', 'level', 'lat', 'lon')
Test V数据已从 E:\huawei\result\cnn_transformer_testy_v_24_5year_order_fusion.nc 加载。维度: ('time', 'level', 'lat', 'lon')
Test W数据已从 E:\huawei\result\cnn_transformer_testy_w_24_5year_order_fusion.nc 加载。维度: ('time', 'level', 'lat', 'lon')
T

C:\Users\TBYC\AppData\Roaming\Python\Python311\site-packages\cfgrib\xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(


正在将Test数据插值到目标空间网格 (level, lat, lon)...
正在计算ECMWF与Test数据的相关系数...

--- ECMWF与Test数据相关系数结果 ---
U分量平均相关系数 (ECMWF vs Test): 0.9404
V分量平均相关系数 (ECMWF vs Test): 0.9727
W分量平均相关系数 (ECMWF vs Test): 0.4776

正在计算ECMWF与Test数据的MAE, RMSE, RMSPE...
U分量 MAE (ECMWF vs Test): 0.8041
U分量 RMSE (ECMWF vs Test): 1.0949
U分量 RMSPE (ECMWF vs Test): 0.0164
V分量 MAE (ECMWF vs Test): 0.8406
V分量 RMSE (ECMWF vs Test): 1.1454
V分量 RMSPE (ECMWF vs Test): 0.0238
W分量 MAE (ECMWF vs Test): 0.0108
W分量 RMSE (ECMWF vs Test): 0.0177
W分量 RMSPE (ECMWF vs Test): 0.0259

--- 正在计算Test与Predict数据的相关系数和误差指标 ---
Predict数据和 (已匹配的) Test数据共找到 104 个共同时间步。
正在将Predict数据插值到共同的空间网格...
正在计算Test与Predict数据的相关系数...

--- Test与Predict数据相关系数结果 ---
U分量平均相关系数 (Test vs Predict): 0.6537
V分量平均相关系数 (Test vs Predict): 0.4574
W分量平均相关系数 (Test vs Predict): 0.2903

正在计算Test与Predict数据的MAE, RMSE, RMSPE...
U分量 MAE (Test vs Predict): 2.9970
U分量 RMSE (Test vs Predict): 4.1380
U分量 RMSPE (Test vs Predict): 0.0621
V分量 MAE (Test vs Predict): 3.0842
V分量 RMSE (Test vs Pred